In [1]:
import os
import torch
import numpy as np
import pandas as pd
from pprint import pprint
from matplotlib import pyplot as plt

from deep_fields.models.random_fields.poisson_covariance import PoissonCovariance, MertonJumpsPoissonCovariance
from torch.distributions import Normal, MultivariateNormal
from torch.distributions import Beta, Bernoulli
from sklearn import metrics


from scipy.stats import invwishart as Invwishart
from scipy.stats import wishart as Wishart

In [2]:
new_sample = True
data_dir = "C:/Users/cesar/Desktop/Projects/General/deep_random_fields/data/raw/merton_poisson_covariance/"
my_data_path = os.path.join(data_dir, "merton_simulation.tr")
if new_sample:
    model_param = MertonJumpsPoissonCovariance.get_parameters()
    model_param.update({"jump_size_a": 5.})
    model_param.update({"jump_size_b": 1.})
    model_param.update({"jump_arrival_alpha": .5,})
    model_param.update({"jump_arrival_beta": .5,})
    model_param.update({"returns_mean_a": 1.,})
    model_param.update({"returns_mean_b": 1.,})
    model_param.update({"diffusion_covariance_normalization": 1.})

    mjpc_s = MertonJumpsPoissonCovariance(None, None, None, **model_param)
    data_loader = mjpc_s.sample()
    data_ = {"data_loader":data_loader,"model_param":model_param}
    torch.save(data_,my_data_path)

New Model Set For Inference
(<class 'AttributeError'>, AttributeError("'NoneType' object has no attribute 'get'"), <traceback object at 0x00000250CA8B56C0>)


In [3]:
inference_param = MertonJumpsPoissonCovariance.get_inference_parameters()
mjpc = MertonJumpsPoissonCovariance(None, data_loader, None, **model_param)
monte_carlo_parameters = mjpc.initialize_inference(data_loader, None, **inference_param)

New Model Set For Inference
Data Loader has No Parameters


In [5]:
inference_param

{'nmc': 1000,
 'burning': 200,
 'metrics_logs': 200,
 'train_diffusion_covariance': False,
 'train_expected_returns': False,
 'train_jumps_arrival': False,
 'train_jumps_size': False,
 'train_jumps_intensity': False,
 'train_jumps_mean': False,
 'train_jumps_covariance': False}

In [6]:
monte_carlo_parameters["diffusion_covariance"]

[array([[1.8378966 , 1.6238575 , 1.692905  , 0.6119352 ],
        [1.6238575 , 1.5547155 , 1.5495377 , 0.52732325],
        [1.692905  , 1.5495377 , 2.7870789 , 0.84775287],
        [0.6119352 , 0.52732325, 0.84775287, 0.32744008]], dtype=float32)]

In [7]:
data_loader["diffusion_covariance"]

tensor([[1.8379, 1.6239, 1.6929, 0.6119],
        [1.6239, 1.5547, 1.5495, 0.5273],
        [1.6929, 1.5495, 2.7871, 0.8478],
        [0.6119, 0.5273, 0.8478, 0.3274]])

In [8]:
inference_param

{'nmc': 1000,
 'burning': 200,
 'metrics_logs': 200,
 'train_diffusion_covariance': False,
 'train_expected_returns': False,
 'train_jumps_arrival': False,
 'train_jumps_size': False,
 'train_jumps_intensity': False,
 'train_jumps_mean': False,
 'train_jumps_covariance': False}

# Gibbs Sampler

In [6]:
jumps_size = torch.Tensor(monte_carlo_values["jumps_size"][-1])  # H
sigma = torch.Tensor(monte_carlo_values["diffusion_covariance"])
arrivals_intensity = monte_carlo_values["arrivals_intensity"][-1]

sigma_inverse = torch.inverse(sigma)

log_returns = data_loader["log_returns"]
interest_rate = data_loader["expected_returns"]

NameError: name 'monte_carlo_values' is not defined

In [ ]:

# probability of arriving
indicator_mean = log_returns - interest_rate[None, :] - jumps_size
indicator_mean = indicator_mean.unsqueeze(-1)
indicator_mean_ = torch.matmul(indicator_mean.transpose(2, 1), sigma_inverse[None, :, :])
indicator_mean = torch.matmul(indicator_mean_, indicator_mean)
indicator_mean.squeeze()
bernoulli_probability_1 = arrivals_intensity * torch.exp(-.5 * indicator_mean).squeeze()

#probability of not arriving
indicator_mean = (log_returns - interest_rate[None, :]).unsqueeze(1)
indicator_mean_ = m(indicator_mean, sigma_inverse[None, :, :])
indicator_mean = -.5*m(indicator_mean_,indicator_mean.permute(0,2,1)).squeeze()
indicator_mean = torch.exp(indicator_mean)
bernoulli_probability_0 = (1. - arrivals_intensity)*indicator_mean

bernoulli_probability = bernoulli_probability_1/(bernoulli_probability_0 + bernoulli_probability_1+EPSILON)

indicator_posterior = Bernoulli(bernoulli_probability)
indicator = indicator_posterior.sample()

monte_carlo_values["arrivals_indicator"].append(indicator.numpy())